## Paper: https://arxiv.org/abs/2202.03772v2

In [1]:
import sys
import os
from tqdm import tqdm
import numpy as np
# load torch modules

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import glob
import gc

In [2]:
sys.path.insert(0, "../../")
from models.ParT.ParticleTransformerEncoder import ParTEncoder
from models.ParT.utils import calculate_cartesian_components, generate_mask
from dataset.ParticleDataset import ParticleDataset

In [3]:
world_size = torch.cuda.device_count()
if world_size:
    device = torch.device("cuda:0")
    for i in range(world_size):
        print(
            f"Device {i}: {torch.cuda.get_device_name(i)}", flush=True
        )
else:
    device = torch.device("cpu")
    print("Device: CPU", file=logfile, flush=True)

Device 0: NVIDIA A100-SXM4-80GB


# Load some data

In [4]:
def load_data(dataset_path, flag, num_jets, return_labels=True):
    dataset_path += f"/{flag}"
    dataset = ParticleDataset(dataset_path, num_jets=num_jets, return_labels=return_labels)
    return dataset

In [5]:
dataset = load_data("/j-jepa-vol/J-JEPA/data/top/ptcl", "val", 100)

Loading /j-jepa-vol/J-JEPA/data/top/ptcl/val/val.h5
Loaded /j-jepa-vol/J-JEPA/data/top/ptcl/val/val.h5
__getitem__ returns ['p3 (px, py, pz)', 'p4 (eta, phi, log_pt, log_e)', 'mask', 'labels']


# Forward pass

In [7]:
mean_log_e, std_log_e = dataset.stats['part_e_log']
log_e = dataset.p4[:, -1, :] * std_log_e + mean_log_e
norm_energy = torch.exp(torch.from_numpy(log_e)) * torch.from_numpy(dataset.mask) 
energy = norm_energy 
v = torch.cat([torch.from_numpy(dataset.p3), energy.view(energy.shape[0], 1, -1)], dim=1).to(device)
print("shape of v:", v.shape)  # Should print (batch_size, 4, 50)

output_dim = 128
fc_params = [(128, 0.05), (256, 0.05), (512, 0.05)]
fc_params = [(128, 0.)]
encoder = ParTEncoder(input_dim=4, embed_dims=[128, 512, output_dim], fc_params=fc_params).to(
    device
)  # the last embedding dimension is also the dimension after attention (before fc projector specified by fc_params)
print(encoder)
mask = torch.from_numpy(dataset.mask).view(dataset.mask.shape[0], 1, -1).to(device, torch.bool)
reps = encoder(torch.from_numpy(dataset.p4).to(device, torch.float32), v.to(torch.float32), mask)
print(reps)
print("shape of latent space:", reps.shape) # Should print (batch_size, output_dim)
del dataset
gc.collect()

shape of v: torch.Size([100, 4, 128])
tensor([[[3.0240, 1.3514, 0.0000,  ..., 0.0000, 0.0000, 1.3728],
         [1.1285, 0.9192, 0.0000,  ..., 0.0000, 0.0000, 0.6837],
         [1.3783, 1.9105, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [1.3494, 0.7728, 0.0000,  ..., 0.0000, 0.0000, 0.3054],
         [0.6055, 0.8452, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[1.8740, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.1576],
         [0.5093, 0.7756, 0.0000,  ..., 0.0000, 0.0000, 1.2010],
         [2.5054, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9615],
         ...,
         [0.8155, 0.1519, 0.0000,  ..., 0.0000, 0.0000, 0.1201],
         [1.6900, 0.8803, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[1.7024, 0.1628, 0.0000,  ..., 0.0000, 0.0000, 0.5405],
         [2.0470, 1.0934, 0.0000,  ..., 0.0000, 0.0000, 0.7562],
         [0.0000, 0.

/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


# Training ParticleTransformerEncoder with a linear classification layer

In [8]:
dataset = load_data("/j-jepa-vol/J-JEPA/data/top/ptcl", "val", 10000)

Loading /j-jepa-vol/J-JEPA/data/top/ptcl/val/val.h5
Loaded /j-jepa-vol/J-JEPA/data/top/ptcl/val/val.h5
__getitem__ returns ['p3 (px, py, pz)', 'p4 (eta, phi, log_pt, log_e)', 'mask', 'labels']


In [9]:
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

def Projector(mlp, embedding):
    mlp_spec = f"{embedding}-{mlp}"
    layers = []
    f = list(map(int, mlp_spec.split("-")))
    for i in range(len(f) - 2):
        layers.append(nn.Linear(f[i], f[i + 1]))
        layers.append(nn.BatchNorm1d(f[i + 1]))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(f[-2], f[-1], bias=False))
    return nn.Sequential(*layers)
proj = Projector(2, fc_params[-1][0])

optimizer = optim.Adam(
            [{"params": proj.parameters()}, {"params": encoder.parameters(), "lr": 1e-6}],
            lr=1e-4,
        )
proj.to(device)
encoder.to(device)
loss = nn.CrossEntropyLoss(reduction="mean")
softmax = torch.nn.Softmax(dim=1)

encoder.train()
proj.train()
num_epochs = 30
dataloader = DataLoader(dataset, batch_size=128, shuffle=False)

for epoch in range(num_epochs):
    loss_e = []
    predicted_e, correct_e = [], []
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        x = batch.p4.to(device)
        mean_log_e, std_log_e = dataset.stats['part_e_log']
        log_e = batch.p4[:, -1, :] * std_log_e + mean_log_e
        energy = torch.exp(log_e) * batch.mask
        v = torch.cat([batch.p3, energy.view(energy.shape[0], 1, -1)], dim=1).to(device)
        mask = batch.mask.view(batch.mask.shape[0], 1, -1).to(device, torch.bool)
        y = batch.labels.to(device)
        # print(x, v)
        reps = encoder(x.to(torch.float32), v.to(torch.float32), mask) # num_ptcls, batch_size, emb_dim
        reps = reps.sum(dim=0) # batch_size, emb_dim
        out = proj(reps)
        batch_loss = loss(out, y.long()).to(device)
        batch_loss.backward()
        optimizer.step()
        batch_loss = batch_loss.detach().cpu().item()
        # print(batch_loss)
        loss_e.append(batch_loss)
        predicted_e.append(softmax(out).cpu().data.numpy())
        correct_e.append(y.cpu().data)
    print(f"Epoch {epoch} loss {np.mean(loss_e)}")
    predicted = np.concatenate(predicted_e)
    target = np.concatenate(correct_e)

    # get the accuracy
    accuracy = accuracy_score(target, predicted[:, 1] > 0.5)
    print(
        "epoch: " + str(epoch) + ", accuracy: " + str(round(accuracy, 5)),
    )

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:09<00:00,  1.13it/s]


Epoch 0 loss 18.387659797185584
epoch: 0, accuracy: 0.5859


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:07<00:00,  1.17it/s]


Epoch 1 loss 5.6282502065731
epoch: 1, accuracy: 0.6536


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:08<00:00,  1.16it/s]


...

Epoch 27 loss 1.0558659879467156
epoch: 27, accuracy: 0.7851


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:08<00:00,  1.16it/s]


Epoch 28 loss 1.072640320922755
epoch: 28, accuracy: 0.7787


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:08<00:00,  1.16it/s]

Epoch 29 loss 1.0657228027717978
epoch: 29, accuracy: 0.7833
